In [1]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()


In [2]:
from fastapi import FastAPI, File, UploadFile
import openai
import fitz  # PyMuPDF for PDF parsing
import docx2txt
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

# FastAPI app
app = FastAPI()

# OpenAI API Key (Replace with your actual API key)
openai.api_key = input("Enter API key: ")
llm = ChatOpenAI(model="gpt-4-turbo", openai_api_key=openai.api_key)

# Function to extract text from resume
def extract_resume_text(file: UploadFile):
    file_content = file.file.read()  # Read the file content
    file.file.seek(0)  # Reset file pointer after reading

    if file.filename.endswith(".pdf"):
        with fitz.open(stream=file_content, filetype="pdf") as doc:
            return " ".join([page.get_text() for page in doc])
    elif file.filename.endswith(".docx"):
        return docx2txt.process(file.file)

    return None

# API Endpoint: Upload Resume
@app.post("/upload_resume/")
async def upload_resume(file: UploadFile = File(...)):
    resume_text = extract_resume_text(file)

    if not resume_text:
        return {"error": "Invalid file format or could not extract text"}

    # LangChain Chatbot Prompt
    messages = [
        SystemMessage(content="You are an expert in resume analysis."),
        HumanMessage(content=f"Analyze this resume and provide feedback:\n{resume_text}")
    ]

    response = llm.invoke(messages)

    return {"analysis": response.content}


Enter API key: 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


In [3]:
import threading

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run).start()


In [5]:
!pkill -f ngrok
from pyngrok import ngrok

public_url = ngrok.connect(8000).public_url
print("Public URL:", public_url)


Public URL: https://f3e8-35-229-43-198.ngrok-free.app


In [ ]:
!kill -9 $(lsof -t -i:8000)

In [1]:
!pkill -f ngrok


News Summarizer


In [2]:
# Install necessary packages
!pip install fastapi uvicorn requests openai langchain langchain_openai pyngrok nest_asyncio

# Imports
import requests
import openai
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from pyngrok import ngrok

# Apply nest_asyncio for running FastAPI in Colab
nest_asyncio.apply()

# Initialize FastAPI
app = FastAPI()

# OpenAI API Key
openai.api_key = input("Enter your OpenAI API key: ")

# Initialize LangChain Model
llm = ChatOpenAI(model="gpt-4-turbo", openai_api_key=openai.api_key)

# Function to fetch news (Example using NewsAPI)
def fetch_news():
    url = "https://newsapi.org/v2/top-headlines?country=us&apiKey=YOUR_NEWSAPI_KEY"
    response = requests.get(url)

    if response.status_code == 200:
        articles = response.json()["articles"]
        news_text = "\n".join([f"{article['title']}: {article['description']}" for article in articles[:5]]) # Fetch top 5
        return news_text
    return "No news found."

# API Endpoint: Get News Summary
@app.get("/summarize_news/")
async def summarize_news():
    news_content = fetch_news()

    if news_content == "No news found.":
        return {"error": "Could not fetch news."}

    # LangChain Chatbot Prompt
    messages = [
        SystemMessage(content="You are an AI that summarizes news articles."),
        HumanMessage(content=f"Summarize these news articles:\n{news_content}")
    ]

    response = llm.invoke(messages)

    return {"summary": response.content}

# Start FastAPI Server
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

import threading
threading.Thread(target=run).start()

# Create a public URL using ngrok
public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")


Enter your OpenAI API key:  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


INFO:     Started server process [16161]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://aeaf-35-229-43-198.ngrok-free.app


In [ ]:
API key   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